In [5]:
pip install pyspark

     -------------------------------------- 316.9/316.9 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 200.5/200.5 kB 11.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425367 sha256=4dd8365476daa79f6ab6e4d1cb36cfa64f188c9c8ba5d1aa418c2e1b284a44e9
  Stored in directory: c:\users\namit\appdata\local\pip\cache\wheels\57\bd\14\ce9e21f2649298678d011fb8f71ed38ee70b42b94fef0be142
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spark-nlp

     ------------------------------------- 547.3/547.3 kB 11.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sparknlp
import pyspark

In [7]:
spark = sparknlp.start()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [7]:
data = spark.createDataFrame([
["""As she traveled across the world, Emma visited many different places 
and met many fascinating people. She walked the busy streets of Tokyo, 
hiked the rugged mountains of Nepal, and swam in the crystal-clear waters 
of the Caribbean. Along the way, she befriended locals like Akira, Rajesh, 
and Maria, each with their own unique stories to tell. Emma's travels took her 
to many cities, including New York, Paris, and Hong Kong, where she savored 
delicious foods and explored vibrant cultures. No matter where she went, 
Emma always found new wonders to discover and memories to cherish."""]
]).toDF('text')

In [8]:
data.show()

+--------------------+
|                text|
+--------------------+
|As she traveled a...|
+--------------------+



In [9]:
person_matches = """
Emma
Akira
Rajesh
Maria
"""

with open('person_matches.txt','w') as f :
    f.write(person_matches)


location_matches = """
Nepal
New York
Tokyo
Caribbean
Paris
Hong Kong
"""

with open("Location_matches.txt",'w') as f:
    f.write(location_matches)

In [21]:
from sparknlp.base import DocumentAssembler , Pipeline , ReadAs
from sparknlp.annotator import (
    Tokenizer,
    TextMatcher
)

from pyspark.sql.types import StringType
import pyspark.sql.functions as F

In [22]:
#step 1 transformation of text from raw to doucment annotation

DocumentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

In [38]:
#step 2 gets the token of text
tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

In [31]:
# step 3 : Person Matcher
person_extractor = TextMatcher() \
    .setInputCols("document","token") \
    .setEntities("person_matches.txt",ReadAs.TEXT) \
    .setEntityValue("PERSON") \
    .setOutputCol("person_entitiy") \
    .setCaseSensitive(False)


In [32]:
# step 4   : Location Matcher
location_extractor = TextMatcher() \
    .setInputCols("document","token") \
    .setEntities("Location_matches.txt",ReadAs.TEXT) \
    .setEntityValue("LOCATION") \
    .setOutputCol("location_entitiy") \
    .setCaseSensitive(False)

In [39]:
pipeline = Pipeline().setStages([
DocumentAssembler,tokenizer,
person_extractor,location_extractor])

In [40]:
results = pipeline.fit(data).transform(data)

In [47]:
results.selectExpr("location_entity.result").show()

AnalysisException: cannot resolve 'location_entity.result' given input columns: [document, location_entitiy, person_entitiy, text, token]; line 1 pos 0;
'Project ['location_entity.result]
+- Project [text#2, document#60, token#68, person_entitiy#74, UDF(array(document#60, token#68)) AS location_entitiy#81]
   +- Project [text#2, document#60, token#68, UDF(array(document#60, token#68)) AS person_entitiy#74]
      +- Project [text#2, document#60, UDF(array(document#60)) AS token#68]
         +- Project [text#2, UDF(text#2) AS document#60]
            +- Project [_1#0 AS text#2]
               +- LogicalRDD [_1#0], false
